### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [123]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [124]:
#Calculate the total number of schools
school_count = school_data_complete["school_name"].nunique()
school_count

15

In [125]:
#Calculate the total number of students
student_count = school_data_complete["student_name"].count()
student_count

39170

In [126]:
#Calculate the Total Budget
#create dataframe for school/budget
df = pd.DataFrame(school_data_complete, columns=["school_name","budget"])

#remove duplicates
df_un = df.drop_duplicates()

#sum resulting budget column
total_budget = df_un['budget'].sum()
total_budget

24649428

In [127]:
#Calculate the average maths score
avg_maths = round(school_data_complete["maths_score"].mean(),6)
avg_maths

70.338192

In [128]:
#Calculate the average reading score
avg_reading = round(school_data_complete["reading_score"].mean(),6)
avg_reading

69.980138

In [129]:
#Calculate the percentage of students with a passing maths score (50 or greater)
#len code sourced and adapted from https://sparkbyexamples.com/pandas/how-to-get-pandas-count-rows-with-condition/?expand_article=1
pass_maths = len(school_data_complete[school_data_complete["maths_score"] >= 50])
pass_maths_perc = round((pass_maths/student_count)*100,6)
pass_maths_perc

86.078632

In [130]:
#Calculate the percentage of students with a passing reading score (50 or greater)
pass_reading = len(school_data_complete[school_data_complete["reading_score"] >= 50])
pass_reading_perc = round((pass_reading/student_count)*100,6)
pass_reading_perc

84.426857

In [131]:
#Calculate the percentage of students who passed maths and reading (% Overall Passing)
pass_mr = len(school_data_complete[(school_data_complete["reading_score"] >= 50) & (school_data_complete["maths_score"] >= 50)])
pass_mr_perc = round((pass_mr/student_count)*100,6)
pass_mr_perc

72.808272

In [132]:
#Create a dataframe to hold the above results
results = pd.DataFrame({'Total Schools': [school_count],'Total Students' : [student_count],'Total Budget' : [total_budget],
                        'Average Maths Score' : [avg_maths],'Average Reading Score' : [avg_reading],'% Passing Maths' : [pass_maths_perc],
                        '% Passing Reading' : [pass_reading_perc],'% Overall Passing' : [pass_mr_perc]})
results

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,39170,24649428,70.338192,69.980138,86.078632,84.426857,72.808272


In [133]:
#Optional: give the displayed data cleaner formatting
results.style.format({
    "Total Budget":"${:,.2f}", 
    "Average Maths Score":"{:,.2f}", 
    "Average Reading Score":"{:,.2f}",
    "% Passing Maths":"{:,.2f}%",
    "% Passing Reading":"{:,.2f}%",
    "% Overall Passing":"{:,.2f}%"
    })

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",70.34,69.98,86.08%,84.43%,72.81%


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [135]:
#Group data by School name
school_groups = school_data_complete.groupby(['school_name'])

In [136]:
#set up static data columns unique to each school by using.first() for type and budget
#code sourced and adapted from https://stackoverflow.com/questions/20067636/pandas-dataframe-get-first-row-of-each-group
sch_type = school_groups["type"].first()
sch_budget = school_groups["budget"].first()

#calculate number of students
sch_stu = school_groups["student_name"].count()

#calculate scores and budget per student
sch_avg_maths = school_groups["maths_score"].mean()
sch_avg_reading = school_groups["reading_score"].mean()
stu_budget = (sch_budget/sch_stu)

#calculate percentages
sch_pass_maths = school_data_complete[school_data_complete["maths_score"] >= 50]
sch_pass_maths_gp = sch_pass_maths.groupby("school_name")
sch_pass_maths_perc = sch_pass_maths_gp["maths_score"].count() / sch_stu *100

sch_pass_read = school_data_complete[school_data_complete["reading_score"] >= 50]
sch_pass_read_gp = sch_pass_read.groupby("school_name")
sch_pass_read_perc = sch_pass_read_gp["reading_score"].count() / sch_stu *100

sch_pass_mr = school_data_complete[(school_data_complete["reading_score"] >= 50) & (school_data_complete["maths_score"] >= 50)]
sch_pass_mr_gp = sch_pass_mr.groupby(['school_name'])
sch_pass_mr_perc = (sch_pass_mr_gp['student_name'].count() / sch_stu) *100

In [137]:
# create dictionary to hold results
sch_summary_result_dict = { "School Type" : sch_type, 
                   "Total Students" : sch_stu, 
                   "Total School Budget" : sch_budget,
                   "Per Student Budget" : stu_budget, 
                   "Average Maths Score" : sch_avg_maths,
                   "Average Reading Score" : sch_avg_reading,
                   "% Passing Maths" : sch_pass_maths_perc,
                   "% Passing Reading" : sch_pass_read_perc,
                   "% Overall Passing" : sch_pass_mr_perc}

In [140]:
#create dataframe from dictionary
sch_summary_result = pd.DataFrame(sch_summary_result_dict)

#format dataframe columns
sch_summary_result["Total School Budget"] = sch_summary_result["Total School Budget"].map("${:,.2f}".format)
sch_summary_result["Average Maths Score"] = sch_summary_result["Average Maths Score"].map("{:,.2f}".format)
sch_summary_result["Average Reading Score"] = sch_summary_result["Average Reading Score"].map("{:,.2f}".format)
sch_summary_result["% Passing Maths"] = sch_summary_result["% Passing Maths"].map("{:,.2f}%".format)
sch_summary_result["% Passing Reading"] = sch_summary_result["% Passing Reading"].map("{:,.2f}%".format)
sch_summary_result["% Overall Passing"] = sch_summary_result["% Overall Passing"].map("{:,.2f}%".format)

sch_summary_result

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",628.0,72.35,71.01,91.64%,87.38%,80.08%
Cabrera High School,Independent,1858,"$1,081,356.00",582.0,71.66,71.36,90.85%,89.07%,80.79%
Figueroa High School,Government,2949,"$1,884,411.00",639.0,68.70,69.08,81.65%,82.81%,67.65%
Ford High School,Government,2739,"$1,763,916.00",644.0,69.09,69.57,82.44%,82.22%,67.47%
Griffin High School,Independent,1468,"$917,500.00",625.0,71.79,71.25,91.21%,88.49%,81.34%
Hernandez High School,Government,4635,"$3,022,020.00",652.0,68.87,69.19,80.95%,81.88%,66.36%
Holden High School,Independent,427,"$248,087.00",581.0,72.58,71.66,89.93%,88.52%,78.92%
Huang High School,Government,2917,"$1,910,635.00",655.0,68.94,68.91,81.69%,81.45%,66.71%
Johnson High School,Government,4761,"$3,094,650.00",650.0,68.84,69.04,82.06%,81.98%,67.19%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,"$1,049,400.00",$583.00,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,72.047762,70.935984,90.797699,87.396849,79.419855


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880


## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

,Year 9,Year 10,Year 11,Year 12
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393
Pena High School,71.996364,72.396000,72.523438,71.187845


## Reading Score by Year

* Perform the same operations as above for reading scores

,Year 9,Year 10,Year 11,Year 12
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521
Pena High School,70.949091,72.324000,71.703125,71.513812


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.36,70.72,88.84,86.39,76.72
$585-630,72.07,71.03,91.52,87.29,79.88
$630-645,69.85,69.84,84.69,83.76,71.00
$645-680,68.88,69.05,81.57,81.77,66.76


## Scores by School Size

* Perform the same operations as above, based on school size.

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348
Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
Large (2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


## Scores by School Type

* Perform the same operations as above, based on school type

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340
